In [105]:
import json
import heapq
import collections
import pandas as pd
from itertools import product
import itertools
from typing import List, Dict, Tuple, Any, Union

In [106]:
# Define helper functions for message passing
def multiplyFactors(factorList: List[Dict], domain: list) -> dict:
    if not factorList:
        return oneFactor(domain) # Return a factor that is one for all assignments
    
    result = factorList[0].copy()
    # print(type(result))
    for factor in factorList[1:]:
        result = multiplyTwoFactors(result, factor)
    return result

def multiplyTwoFactors(factor1 : Dict, factor2 : Dict) -> Dict:
    result = {}
    for assignment1, value1 in factor1.items():
        for assignment2, value2 in factor2.items():
            merged = unifyAssignments(assignment1, assignment2)
            if merged is not None:
                result[merged] = result.get(merged, 0.0) + (value1 * value2)   
    return result

def unifyAssignments(assignment1 : tuple, assignment2 : tuple) -> Union[tuple, None]:
    mergedDict = dict(assignment1)
    for variable, value in assignment2:
        if variable in mergedDict and mergedDict[variable] != value:
            return None
        mergedDict[variable] = value
    return tuple(sorted(mergedDict.items()))

def oneFactor(domain : list) -> dict:
    # Make all possible assignments
    domainList = sorted(domain)
    allAssignments = {}

    for assignmentTuple in itertools.product([0, 1], repeat = len(domainList)):
        assignment = tuple(zip(domainList, assignmentTuple))
        allAssignments[assignment] = 1.0
    return allAssignments
    

def sumOut(factor: Dict[Tuple[Tuple[int, int], ...], float], variablesToSumOut: List[int], domain: List[int]) -> Dict[Tuple[Tuple[int, int], ...], float]:
    # Summation over a specified variables in the factor
    result = {}
    print(factor)
    for assignment, value in factor:
        newAssignment = tuple((var, val) for var, val in assignment if var not in variablesToSumOut)
        result[newAssignment] = result.get(newAssignment, 0) + value
    return result

# Helper Functions to test the implementation
def printUndirectedGraph(graph):
    print("Undirected Graph:")
    for node, neighbors in graph.items():
        print(f"{node}: {sorted(neighbors)}")

def printTriangulatedGraph(graph):
    print("Triangulated Graph:")
    for node, neighbors in graph.items():
        print(f"{node}: {sorted(neighbors)}")

import pandas as pd
from itertools import product

def convert_list_to_dataframe(clique, potential_list):
    """
    Converts a potential given as a list-of-lists into a pandas DataFrame.
    
    Assumes:
      - clique is a set (or frozenset) of variables.
      - Each variable is binary.
      - potential_list is a list containing one inner list of 2^(|clique|) numbers.
      - The ordering of the potential values corresponds to the assignments
        produced by product([0, 1], repeat=len(sorted(clique))).
        
    Returns a DataFrame with one column per variable (using sorted order)
    and a 'value' column for the potential values.
    """
    sorted_vars = sorted(clique)
    num_vars = len(sorted_vars)
    expected_length = 2 ** num_vars
    
    if len(potential_list) == 1 and len(potential_list[0]) == expected_length:
        values = potential_list[0]
        assignments = list(product([0, 1], repeat=num_vars))
        data = []
        for assign, val in zip(assignments, values):
            row = dict(zip(sorted_vars, assign))
            row["value"] = val
            data.append(row)
        return pd.DataFrame(data)
    else:
        raise ValueError("Potential format not supported. Expected a list with one inner list of length 2^(|clique|).")

def join_two_tables(df1, df2):
    """
    Join two factor tables df1 and df2.
    The join is performed on the common variable columns (excluding 'value').
    After joining, the potential values (in the 'value' column) are multiplied.
    """
    keys = list((set(df1.columns) - {"value"}) & (set(df2.columns) - {"value"}))
    if keys:
        merged = pd.merge(df1, df2, on=keys)
    else:
        merged = df1.assign(tmp=1).merge(df2.assign(tmp=1), on="tmp").drop("tmp", axis=1)
    
    if "value_x" in merged.columns and "value_y" in merged.columns:
        merged["value"] = merged["value_x"] * merged["value_y"]
        merged = merged.drop(columns=["value_x", "value_y"])
    return merged

def table_multiply(tables):
    """
    Multiply a list of tables (factors) by successive join operations.
    """
    if not tables:
        return None
    result = tables[0]
    for t in tables[1:]:
        result = join_two_tables(result, t)
    return result

def marginalize_table(df, vars_to_keep):
    """
    Marginalize (sum out) all variables not in vars_to_keep.
    If vars_to_keep is empty, returns a single constant factor.
    """
    if not vars_to_keep:
        total = df["value"].sum()
        return pd.DataFrame({"value": [total]})
    grouped = df.groupby(list(vars_to_keep), as_index=False)["value"].sum()
    return grouped

In [107]:
class Inference:
    def __init__(self, data : dict):
        """
        Initialize the Inference class with the input data.
        
        Parameters:
        -----------
        data : dict
            The input data containing the graphical model details, such as variables, cliques, potentials, and k value.
        
        What to do here:
        ----------------
        - Parse the input data and store necessary attributes (e.g., variables, cliques, potentials, k value).
        - Initialize any data structures required for triangulation, junction tree creation, and message passing.
        
        Refer to the sample test case for the structure of the input data.
        """
        self.testcaseNumber = data.get("TestCaseNumber", 0)
        self.variableCount = data.get("VariablesCount", 0)
        self.potentialCount = data.get("Potentials_count", 0)
        # List of cliques (each clique is a dictionary)
        self.cliquesAndPotentials = data.get("Cliques and Potentials", []) 
        self.k = data.get("k value (in top k)", 0)
        self.z = 0

        # Build the graph from the cliques. Nodes are labelled from 0 to variableCount - 1
        self.undirectedGraph = {i: set() for i in range(self.variableCount)}
        for clique in self.cliquesAndPotentials:
            nodes = clique.get("cliques", [])
            # For each clique, add edges between each pair of nodes
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    self.undirectedGraph[nodes[i]].add(nodes[j])
                    self.undirectedGraph[nodes[j]].add(nodes[i])
        
        # Define the variables needed to construct the triangulated graph
        self.triangulatedGraph = {}
        self.maximalCliques = []

        # Define the variables needed to construct the junction tree
        self.junctionTree = []

        # Make a dictionary to store the potentials for each clique
        self.cliquePotentials = {}


    def triangulate_and_get_cliques(self):
        """
        Triangulate the undirected graph and extract the maximal cliques.
        
        What to do here:
        ----------------
        - Implement the triangulation algorithm to make the graph chordal.
        - Extract the maximal cliques from the triangulated graph.
        - Store the cliques for later use in junction tree creation.

        Refer to the problem statement for details on triangulation and clique extraction.
        """
        temporaryGraph = {n : set(self.undirectedGraph[n]) for n in self.undirectedGraph}
        chordalGraph = {n : set(self.undirectedGraph[n]) for n in self.undirectedGraph}

        # Initialize the degree info into a min heap
        degreeInfo = {node : len(temporaryGraph[node]) for node in temporaryGraph}
        # Initialize the heap
        degreeHeap = [(degreeInfo[node], node) for node in degreeInfo]
        heapq.heapify(degreeHeap)
        # Maintain a set for the nodes that have been eliminated
        eliminatedNodes = set()
        # Store the cliques from each elimination step
        candidateCliques = []

        # Eliminate the nodes one by one to triangulate the graph
        while degreeHeap:
            currDegree, currNode = heapq.heappop(degreeHeap)
            if currNode in eliminatedNodes or degreeInfo[currNode] != currDegree:
                continue

            # Mark the node as eliminated
            eliminatedNodes.add(currNode)
            # Get the neighbours of the node that are not eliminated
            neighbours = temporaryGraph[currNode] - eliminatedNodes
            # Record the clique as the current node and the neighbours
            currClique = set(neighbours)
            currClique.add(currNode)
            candidateCliques.append(currClique)

            # Add fill edges among the neighours in the chordal graph
            for i in neighbours:
                for j in neighbours:
                    # Add edge in the chordal graph if it is not already present
                    if i == j:
                        continue
                    if j not in chordalGraph[i]:
                        chordalGraph[i].add(j)
                        chordalGraph[j].add(i)
                    # Add edge in the temporary graph if it is not already present
                    if j not in temporaryGraph[i]:
                        temporaryGraph[i].add(j)
                        temporaryGraph[j].add(i)
                        degreeInfo[i] += 1
                        degreeInfo[j] += 1
            
            # Remove the current node from the neighbours
            for neighbour in neighbours:
                temporaryGraph[neighbour].discard(currNode)
            temporaryGraph[currNode].clear()
            # Update the degree info
            degreeInfo[currNode] = 0

            # Update the heap with the new degrees
            for neighbour in neighbours:
                heapq.heappush(degreeHeap, (degreeInfo[neighbour], neighbour))
        
        # After exiting the while loop the graph is now triangulated
        self.triangulatedGraph = chordalGraph
        # Extract the maximal cliques from the candidate cliques by removing the duplicates and subsets
        candidateCliques = list(set(frozenset(clique) for clique in candidateCliques))
        finalCliques = []
        for clique in candidateCliques:
            if not any(clique < others for others in candidateCliques if clique != others):
                finalCliques.append(clique)
        self.maximalCliques = [set(clique) for clique in finalCliques]

    
    def get_junction_tree(self):
        """
        Construct the junction tree from the maximal cliques.
        
        What to do here:
        ----------------
        - Create a junction tree using the maximal cliques obtained from the triangulated graph.
        - For each pair of cliques, compute the common variables.
          Then define the directed separator sets:
              S_ij = clique_i - (clique_i ∩ clique_j)
              S_ji = clique_j - (clique_i ∩ clique_j)
        - Use the size of the common set as the weight to construct a maximum spanning tree.
        - Store the junction tree as a list of tuples:
              (clique_i, clique_j, S_ij, S_ji)
          where S_ij is the separator when a message is passed from clique i to clique j,
          and S_ji is for the reverse direction.
        """
        # Build the weighted graph among the cliques
        cliques = self.maximalCliques
        cliqueCount = len(cliques)

        if cliqueCount <= 1:
            self.junctionTree = []
            return
        
        # Collect the edges of the graph as (weight, i, j) where i and j are the indices of the cliques
        weightedEdges = []
        for i in range(cliqueCount):
            for j in range(i + 1, cliqueCount):
                intersection = cliques[i].intersection(cliques[j])
                weight = len(intersection)
                # Add the edge only if the intersection is non-empty
                if weight > 0:
                    # Store the negative weight to use the min heap as a max heap
                    weightedEdges.append((-weight, i, j))
        
        # If there are no edges, then the junction tree is empty
        if not weightedEdges and cliqueCount > 0:
            # Implies that original graph has no intersecting cliques
            self.junctionTree = []
            return
        
        # Initialize the heap with the edges
        heapq.heapify(weightedEdges)

        # Initialize the union-find set to keep track of the connected components
        parent = list(range(cliqueCount))
        rank = [0] * cliqueCount

        # Define the find and union functions for the union-find set
        def find(x):
            if parent[x] != x:
                parent[x] = find(parent[x])
            return parent[x]

        def union(x, y):
            rootX, rootY = find(x), find(y)
            if rootX == rootY:
                return False
            # Union by Rank
            if rank[rootX] > rank[rootY]:
                parent[rootY] = rootX
            elif rank[rootX] < rank[rootY]:
                parent[rootX] = rootY
            else:
                parent[rootY] = rootX
                rank[rootX] += 1
            return True
        
        # Initialize the junction tree as a list of tuples
        spanningTreeEdges = []
        components = cliqueCount
        while weightedEdges and components > 1:
            weight, i, j = heapq.heappop(weightedEdges)
            if union(i, j):
                components -= 1
                spanningTreeEdges.append((i, j, -weight))
        
        # Define the directed separator sets for each edge in the junction tree
        junctionTree = []
        for i, j, weight in spanningTreeEdges:
            intersection = cliques[i].intersection(cliques[j])
            separatorIJ = cliques[i] - intersection
            separatorJI = cliques[j] - intersection
            junctionTree.append((cliques[i], cliques[j], separatorIJ, separatorJI))
            junctionTree.append((cliques[j], cliques[i], separatorJI, separatorIJ))
        
        self.junctionTree = junctionTree

    
    def assign_potentials_to_cliques(self):
        """
        Assign potentials to the cliques in the junction tree.
        
        What to do here:
        ----------------
        - Map the given potentials (from the input data) to the corresponding cliques in the junction tree.
        - Ensure the potentials are correctly associated with the cliques for message passing.
        
        Refer to the sample test case for how potentials are associated with cliques.
        """
        # Convert the maximal cliques into a set of frozensets for easy comparison
        frozenMaximalCliques = [frozenset(clique) for clique in self.maximalCliques]
        self.cliquePotentials = {frozenClique: [] for frozenClique in frozenMaximalCliques}

        # Iterate over each potential from the input data and assign it to the corresponding clique
        for potentialDict in self.cliquesAndPotentials:
            variableList = potentialDict.get("cliques", [])
            potentialValues = potentialDict.get("potentials", [])
            frozenVariableList = frozenset(variableList)
            # Find all the maximal cliques containing the variables in the potential
            for frozenClique in frozenMaximalCliques:
                if frozenVariableList.issubset(frozenClique):
                    self.cliquePotentials[frozenClique].append(potentialValues)
    
    def get_z_value(self):
        junction_tree = self.junctionTree
        potentials = self.cliquePotentials
        """
        Computes the partition function Z from the calibrated junction tree.
        
        Process:
        1. Run message passing to compute messages between cliques.
        2. Choose an arbitrary clique as the 'root'.
        3. Compute the belief for the root clique by multiplying its own potential with all incoming messages.
        4. Marginalize (sum out) all variables in the root clique to obtain a single number Z.
        
        Debug output:
        - Prints the belief for the root clique before summing out all variables.
        - Prints the final partition function Z.
        """
        # Run message passing to compute all messages.
        messages = message_passing(junction_tree, potentials)
        
        # Choose an arbitrary clique as the root (e.g. the first clique in the potentials dictionary).
        root = frozenset(next(iter(potentials)))
        
        # Determine all neighbors of the root using the junction tree.
        neighbors = set()
        for (sender, recipient, S_ij, S_ji) in junction_tree:
            sender_f = frozenset(sender)
            recipient_f = frozenset(recipient)
            if sender_f == root:
                neighbors.add(recipient_f)
            elif recipient_f == root:
                neighbors.add(sender_f)
        
        # Build the belief for the root clique by multiplying its own potential with all incoming messages.
        root_potential = convert_list_to_dataframe(root, potentials[root])
        factors = [root_potential]
        for neighbor in neighbors:
            if (neighbor, root) in messages:
                factors.append(messages[(neighbor, root)])
        belief = table_multiply(factors)
        print("Debug: Belief for root clique {} before summing out all variables:\n{}".format(root, belief))
        
        # Sum over all variables in the belief to obtain the partition function Z.
        Z_table = marginalize_table(belief, set())
        self.z = int(Z_table["value"].iloc[0])
        print("Partition Function Z =", Z)
        return self.z
        

    def convert_flat_potential_to_dataframe(clique, pot_list):
        """
        Converts a flattened potential into a DataFrame.
        
        Parameters:
        - clique: an iterable of variables in the clique.
        - pot_list: a list-of-lists, expected to contain a single list.
            For a clique with n variables, the inner list should have 2^n entries.
        
        Returns:
        - A pandas DataFrame with columns corresponding to sorted(clique) and a 'value' column.
            Each row represents one assignment in lexicographic order (assuming binary variables).
        """
        sorted_vars = sorted(clique)
        num_assignments = 2 ** len(sorted_vars)
        if len(pot_list) != 1 or len(pot_list[0]) != num_assignments:
            raise ValueError(
                f"Potential for clique {clique} must be provided as a single row with {num_assignments} entries."
            )
        potential_values = pot_list[0]
        assignments = list(itertools.product([0, 1], repeat=len(sorted_vars)))
        data = [list(assignment) + [val] for assignment, val in zip(assignments, potential_values)]
        columns = sorted_vars + ['value']
        return pd.DataFrame(data, columns=columns)
    
    def compute_marginals(self):
        self.variables = sorted({var for clique in self.cliquePotentials for var in clique})
        """
        Compute the marginal probabilities for all variables in the graphical model.
        
        Uses the provided self.cliquePotentials (in flattened format, e.g.,
        {frozenset({0, 1}): [[3, 4, 5, 6]], ...})
        and self.junctionTree to perform message passing until convergence.
        
        Returns:
        - A list of lists, where each inner list contains the marginal probabilities
            for one variable (ordered by sorted(self.variables)).
        """
        # Run message passing using the clique potentials and the junction tree.
        messages = message_passing(self.junctionTree, self.cliquePotentials)
        
        # Build the neighbors dictionary from self.junctionTree.
        neighbors = {}
        for (sender, recipient, S_ij, S_ji) in self.junctionTree:
            sender_f = frozenset(sender)
            recipient_f = frozenset(recipient)
            neighbors.setdefault(sender_f, set()).add(recipient_f)
            neighbors.setdefault(recipient_f, set()).add(sender_f)
        
        # Compute calibrated beliefs for each clique.
        beliefs = {}
        # Here we use the keys from self.cliquePotentials as the cliques.
        for clique in self.cliquePotentials.keys():
            clique_f = frozenset(clique)
            # Convert the flattened potential to a DataFrame.
            factors = [convert_flat_potential_to_dataframe(clique, self.cliquePotentials[clique])]
            # Multiply in incoming messages from all neighboring cliques.
            for neighbor in neighbors.get(clique_f, []):
                if (neighbor, clique_f) in messages:
                    factors.append(messages[(neighbor, clique_f)])
            beliefs[clique_f] = table_multiply(factors)
        
        # For each variable, choose a clique that contains it and marginalize the belief to obtain its marginal.
        marginals = {}
        for var in self.variables:
            for clique_f, belief in beliefs.items():
                if var in clique_f:
                    marginals[var] = marginalize_table(belief, {var})
                    break
    
        # Convert each marginal table (for a single variable) into a list of probabilities.
        ordered_vars = sorted(self.variables)
        marginals_list = [convert_table_to_list(marginals[var]) for var in ordered_vars]
        
        # Divide each probability by self.z for normalization.
        normed_marginals = [[prob / self.z for prob in marg] for marg in marginals_list]
        
        return normed_marginals
    
    def compute_top_k(self):
        """
        Compute the top-k most probable assignments in the graphical model.
        
        What to do here:
        ----------------
        - Use the message passing algorithm to find the top-k assignments with the highest probabilities.
        - Return the assignments along with their probabilities in the specified format.
        
        Refer to the sample test case for the expected format of the top-k assignments.
        """
        pass






In [108]:
def message_passing(junction_tree, potentials):
        """
        Implements message passing on a junction tree.
        
        Parameters:
        - junction_tree: a list of tuples (sender, recipient, S_ij, S_ji)
            where each element is a set of variables.
        - potentials: a dictionary mapping each clique (as a frozenset) to its potential,
            where each potential is given as a list-of-list.
        
        The algorithm sends a message m_ij from node i to node j once node i has received 
        messages from all neighbors except j. The message is computed by multiplying the node’s
        potential with incoming messages and then marginalizing out the separator S_ij.
        
        Debug print statements:
        - Before marginalization: prints the product table (over all variables in the clique).
        - After marginalization: prints the final message table m_ij (defined over sender_f \ S_ij).
        
        Returns:
        - messages: a dictionary with keys (i, j) (both as frozensets) and values being the message tables.
        """
        # Convert each potential from list-of-lists into a DataFrame.
        potentials_new = {}
        for clique, pot_list in potentials.items():
            potentials_new[frozenset(clique)] = convert_list_to_dataframe(clique, pot_list)
        
        # Build the neighbor dictionary from the junction tree.
        neighbors = {}
        for (sender, recipient, S_ij, S_ji) in junction_tree:
            sender_f = frozenset(sender)
            recipient_f = frozenset(recipient)
            if sender_f not in neighbors:
                neighbors[sender_f] = set()
            if recipient_f not in neighbors:
                neighbors[recipient_f] = set()
            neighbors[sender_f].add(recipient_f)
            neighbors[recipient_f].add(sender_f)
        
        messages = {}
        updated = True
        while updated:
            updated = False
            for (sender, recipient, S_ij, S_ji) in junction_tree:
                sender_f = frozenset(sender)
                recipient_f = frozenset(recipient)
                
                if (sender_f, recipient_f) in messages:
                    continue
                
                ready = True
                for neighbor in neighbors[sender_f]:
                    if neighbor == recipient_f:
                        continue
                    if (neighbor, sender_f) not in messages:
                        ready = False
                        break
                if not ready:
                    continue
                
                # Gather factors: the sender's own potential and all incoming messages (except from the recipient)
                factors = [potentials_new[sender_f]]
                for neighbor in neighbors[sender_f]:
                    if neighbor == recipient_f:
                        continue
                    factors.append(messages[(neighbor, sender_f)])
                
                product_table = table_multiply(factors)
                print("Debug: Product table for node {} before marginalization:\n{}".format(sender_f, product_table))
                
                # Instead of keeping the separator S_ij, marginalize out S_ij, i.e. keep sender_f \ S_ij.
                keep_vars = sender_f - S_ij
                message_table = marginalize_table(product_table, keep_vars)
                print("Debug: Message from node {} to node {} after marginalizing out {} (keeping {}):\n{}".format(
                    sender_f, recipient_f, S_ij, keep_vars, message_table))
                
                messages[(sender_f, recipient_f)] = message_table
                updated = True

        print("\nFinal messages:")
        for key, msg in messages.items():
            print("Message from node {} to node {}:\n{}".format(key[0], key[1], msg))
        return messages

In [109]:
# Read in the data from the json file 
path = 'Sample_Testcase.json'
with open(path, 'r') as file:
    data = json.load(file)
sampleInference = Inference(data[0]['Input'])
sampleInference.triangulate_and_get_cliques()
sampleInference.get_junction_tree()
sampleInference.assign_potentials_to_cliques()
print(sampleInference.cliquePotentials)
Z = sampleInference.get_z_value()
marginalz = sampleInference.compute_marginals()


{frozenset({0, 1}): [[3, 4, 5, 6]], frozenset({1, 2}): [[2, 7, 1, 3]], frozenset({0, 3}): [[5, 8, 2, 7]]}
Debug: Product table for node frozenset({1, 2}) before marginalization:
   1  2  value
0  0  0      2
1  0  1      7
2  1  0      1
3  1  1      3
Debug: Message from node frozenset({1, 2}) to node frozenset({0, 1}) after marginalizing out {2} (keeping frozenset({1})):
   1  value
0  0      9
1  1      4
Debug: Product table for node frozenset({0, 1}) before marginalization:
   0  1  value
0  0  0     27
1  1  0     45
2  0  1     16
3  1  1     24
Debug: Message from node frozenset({0, 1}) to node frozenset({0, 3}) after marginalizing out {1} (keeping frozenset({0})):
   0  value
0  0     43
1  1     69
Debug: Product table for node frozenset({0, 3}) before marginalization:
   0  3  value
0  0  0      5
1  0  1      8
2  1  0      2
3  1  1      7
Debug: Message from node frozenset({0, 3}) to node frozenset({0, 1}) after marginalizing out {3} (keeping frozenset({0})):
   0  value


In [110]:
########################################################################

# Do not change anything below this line

########################################################################

class Get_Input_and_Check_Output:
    def __init__(self, file_name):
        with open(file_name, 'r') as file:
            self.data = json.load(file)
    
    def get_output(self):
        n = len(self.data)
        output = []
        for i in range(n):
            inference = Inference(self.data[i]['Input'])
            inference.triangulate_and_get_cliques()
            inference.get_junction_tree()
            inference.assign_potentials_to_cliques()
            z_value = inference.get_z_value()
            marginals = inference.compute_marginals()
            top_k_assignments = inference.compute_top_k()
            output.append({
                'Marginals': marginals,
                'Top_k_assignments': top_k_assignments,
                'Z_value' : z_value
            })
        self.output = output

    def write_output(self, file_name):
        with open(file_name, 'w') as file:
            json.dump(self.output, file, indent=4)


if __name__ == '__main__':
    evaluator = Get_Input_and_Check_Output('Sample_Testcase.json')
    evaluator.get_output()
    evaluator.write_output('Sample_Testcase_Output.json')

Debug: Product table for node frozenset({1, 2}) before marginalization:
   1  2  value
0  0  0      2
1  0  1      7
2  1  0      1
3  1  1      3
Debug: Message from node frozenset({1, 2}) to node frozenset({0, 1}) after marginalizing out {2} (keeping frozenset({1})):
   1  value
0  0      9
1  1      4
Debug: Product table for node frozenset({0, 1}) before marginalization:
   0  1  value
0  0  0     27
1  1  0     45
2  0  1     16
3  1  1     24
Debug: Message from node frozenset({0, 1}) to node frozenset({0, 3}) after marginalizing out {1} (keeping frozenset({0})):
   0  value
0  0     43
1  1     69
Debug: Product table for node frozenset({0, 3}) before marginalization:
   0  3  value
0  0  0      5
1  0  1      8
2  1  0      2
3  1  1      7
Debug: Message from node frozenset({0, 3}) to node frozenset({0, 1}) after marginalizing out {3} (keeping frozenset({0})):
   0  value
0  0     13
1  1      9
Debug: Product table for node frozenset({0, 1}) before marginalization:
   0  1  v